## Chapter 7: Advanced topics

### 7.1 Psychoacoustic modelling

#### 7.1.1 Spectral analysis

In [1]:
import numpy as np

# 예시 음성 신호
speech = np.random.randn(256)  # 256개의 랜덤 신호 예시

# 해밍 창 적용
windowed_signal = np.hamming(256) * speech

# FFT 계산 및 절대값 구하기
S = np.abs(np.fft.fft(windowed_signal))

# 첫 128개의 값 선택
S = S[:128]

# 출력 확인 (첫 10개의 값 확인)
print(S[:10])

[ 7.12244387  9.80079512  4.23067273  5.42473928 11.92033886 13.58198111
 12.25475975  6.1852314   1.90784478  8.3483541 ]


#### 7.1.4 Equal-loudness pre-emphasis

In [2]:
# 기존에 설계한 f2bark, bark2f 함수 재호출 코드
# f2bark 함수 정의 (Hz -> Bark 변환)
def f2bark(hz):
    cn = 2 * np.pi * hz / (1200 * np.pi)
    bark = 6 * np.log(cn + np.sqrt(cn ** 2 + 1))
    return bark

# bark2f 함수 정의 (Bark -> Hz 변환)
def bark2f(bark):
    hz = 600 * np.sinh(bark / 6)
    return hz

# 교재 본문에 없는 주어진 값 설정

X = np.zeros(40)  # 40개의 Bark 밴드 가정
p = np.random.rand(128)  # 임의의 값으로 신호 스펙트럼 예시 (p)
c = 340  # 예시로 음속 설정

In [3]:
# Eql 계산
Eql = np.zeros(128)
for k in range(1, 129):
    w = 4000 * 2 * np.pi * k / 128
    Eql[k - 1] = (((56.8 * 10 ** 6) + w ** 2) * (w ** 4)) / ((((6.3 * 10 ** 6) + w ** 2) ** 2) * ((0.38 * 10 ** 9) + w ** 2))

In [4]:
# Bark 변환을 위한 4kHz 및 0Hz에 대한 Bark 주파수
b4k = f2bark(4000)
b0 = f2bark(0)
n = 128  # 분석할 스펙트럼 크기

# Bark 해상도 40밴드로 가정
for bi in range(1, 41):
    bark = b0 + bi * (b4k - b0) / 40
    wm = int(round(n * bark2f(bark) / 4000))
    
    if wm == 0:
        wm = 1

    # 하한과 상한 주파수 계산
    w_low = bark2f(bark - 1.3) * 2 * np.pi
    w_hig = bark2f(bark + 2.5) * 2 * np.pi
    wl = int(np.fix(w_low / (4000 * 2 * np.pi / n)))
    wh = int(np.fix(w_hig / (4000 * 2 * np.pi / n)))
    
    # 인덱스 범위 제한
    if wl < 1:
        wl = 1
    if wh > n:
        wh = n

    # 합 계산 수행
    for wi in range(wl, wh + 1):
        w = wi * 2 * np.pi * 4000 / n
        # pi 값 계산
        vlu = 6 * np.log((w / c) + np.sqrt((w / c) ** 2 + 1))
        vlu -= bark
        
        # pi에 따른 가중치 계산
        if vlu < -1.3:
            mul = 0
        elif -1.3 <= vlu <= -0.5:
            mul = 10 ** (2.5 * (vlu + 0.5))
        elif -0.5 < vlu < 0.5:
            mul = 1
        elif 0.5 <= vlu <= 2.5:
            mul = 10 ** (0.5 - vlu)
        else:
            mul = 0
        
        X[bi - 1] += Eql[wm - 1] * mul * p[wi - 1]

# 결과 출력 (X의 첫 10개 밴드 확인)
print(X[:10])

[0.00000000e+00 0.00000000e+00 2.38837626e-05 1.30264490e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


### 7.7 Stereo encoding

In [5]:
# 기존에 설계한 tonegen 함수 재호출 코드
import sounddevice as sd

def tonegen(Ft, Fs, Td):
    # 시간 벡터 생성
    t = np.arange(1, Fs * Td + 1) / Fs
    # 사인파 생성
    s = np.sin(2 * np.pi * Ft * t)
    return s

In [6]:
# 파라미터 설정
Fs = 44100  # 샘플링 속도
Ft = 440    # 주파수 (A4 음)
note = tonegen(Ft, Fs, 0.5)  # 0.5초 동안 440Hz의 음을 생성

# 두 채널의 스테레오 신호 생성
s1 = np.vstack([np.concatenate([np.zeros(20), note]), np.concatenate([note, np.zeros(20)])])
s2 = np.fliplr(s1)  # 좌우 반전된 신호

# 사운드 재생
sd.play(s1.T, Fs)  # 스테레오 신호 재생 (전치 행렬 사용)
sd.wait()

sd.play(s2.T, Fs)  # 좌우 채널이 반전된 신호 재생
sd.wait()

sd.play(s1.T, Fs)  # 원래 신호 재생 (전치 없이)
sd.wait()

#### 7.7.1 Stereo and noise

In [7]:
r1 = 0.2 * np.random.rand(int(Fs * 0.5))  # 0.5초 길이의 임의 신호 (r1)
r2 = 0.2 * np.random.rand(int(Fs * 0.5))  # 0.5초 길이의 임의 신호 (r2)

# 좌우 채널로 동일한 신호 재생 (r1)
stereo_signal_1 = np.vstack([r1, r1])

# 사운드 재생 (좌우 동일 신호)
sd.play(stereo_signal_1.T, samplerate=Fs)  # 전치행렬로 재생
sd.wait()

# 전치된 신호를 사용하여 재생 (r1')
stereo_signal_2 = np.vstack([r1.T, r1.T])

# 사운드 재생 (좌우 동일 신호, 전치된 배열)
sd.play(stereo_signal_2.T, samplerate=Fs)  # 전치행렬로 재생
sd.wait()

In [8]:
# 첫 번째 신호 재생 (r1과 r2 좌우 채널로 설정)
stereo_signal_1 = np.vstack([r1, r2])

# 사운드 재생 (좌우 채널로 r1과 r2 재생)
sd.play(stereo_signal_1.T, samplerate=Fs)
sd.wait()

# 두 번째 신호 재생 (r1'과 r2' 좌우 채널로 설정)
stereo_signal_2 = np.vstack([r1, r2])

# 사운드 재생 (좌우 채널로 전치된 r1과 r2 재생)
sd.play(stereo_signal_2.T, samplerate=Fs)
sd.wait()

#### 7.7.2 Stereo placement

In [9]:
import time

# 파라미터 설정
d = 5  # 거리 (m)
h = 0.1  # 머리 반경 (m)
Fs = 44100  # 샘플링 속도
Ft = 600  # 주파수 (Hz)
# 노트 생성 (기본 톤과 2배 주파수 톤의 합)
note = tonegen(Ft, Fs, 0.1) + tonegen(Ft * 2, Fs, 0.1)

Vs = 350  # 음속 (m/s)
ln = len(note)

# 코사인 법칙 상수
b2c2 = d**2 + h**2
b2c = 2 * d * h

# theta 값으로 좌우 경로 차이 계산 및 재생
for theta in np.arange(-np.pi, np.pi + np.pi/20, np.pi/20):
    # 경로 차이 계산
    lp = b2c2 + b2c * np.cos((np.pi/2) + theta)
    rp = b2c2 + b2c * np.cos((np.pi/2) - theta)
    
    # 사운드 이동 시간 계산
    lt = lp / Vs
    rt = rp / Vs
    
    # 샘플 단위로 시간 계산
    ls = int(round(Fs * lt))
    rs = int(round(Fs * rt))
    
    # 각 채널의 사운드 처리
    if rs > ls:  # 오른쪽이 더 멀리 있는 경우
        df = rs - ls
        left = np.concatenate([note, np.zeros(df)]) / ls
        right = np.concatenate([np.zeros(df), note]) / rs
    else:  # 왼쪽이 더 멀리 있는 경우
        df = ls - rs
        left = np.concatenate([np.zeros(df), note]) / ls
        right = np.concatenate([note, np.zeros(df)]) / rs
    
    # 스테레오 오디오 생성
    audio = np.vstack([left, right])
    
    # 사운드 재생
    sd.play(audio.T, samplerate=Fs)
    sd.wait()
    time.sleep(0.1)  # 약간의 지연 추가

### 7.8 Formant strengthening and steering

In [10]:
def lspnarrow(lsp, sc):
    p = len(lsp)
    wid = np.diff(lsp)
    
    # 가장 좁은 3개의 LSP 쌍을 찾기 위한 초기값 설정
    n = [np.pi, np.pi, np.pi]
    ni = [0, 0, 0]
    
    for lp in range(p - 1):
        if wid[lp] < n[2]:
            if wid[lp] < n[1]:
                if wid[lp] < n[0]:
                    n = [wid[lp], n[0], n[1]]
                    ni = [lp, ni[0], ni[1]]
                else:
                    n[1], n[2] = wid[lp], n[1]
                    ni[1], ni[2] = lp, ni[1]
            else:
                n[2] = wid[lp]
                ni[2] = lp
    
    # 가장 좁은 3개의 LSP 쌍을 더 좁힘
    nlsp = np.copy(lsp)
    for k in range(3):
        nlsp[ni[k]] = lsp[ni[k]] + n[k] * sc
        nlsp[ni[k] + 1] = lsp[ni[k] + 1] - n[k] * sc
    
    return nlsp

# 예시로 LSP 배열과 scale factor 설정
lsp = np.array([0.1, 0.2, 0.5, 0.7, 0.9, 1.2])  # 예시 LSP 값
sc = 0.1  # scale factor

# 함수 호출
nlsp = lspnarrow(lsp, sc)
print("Narrowed LSP:", nlsp)

Narrowed LSP: [0.11 0.19 0.52 0.72 0.88 1.2 ]


#### 7.8.1 Perceptual formant steering

In [11]:
# 예시 LSP 배열과 b 설정
lsp = np.array([100, 200, 300, 400, 500])  # 예시 LSP 값
b = 0.1  # Bark 스케일에 더할 값

# 변환 작업
lsp2 = np.zeros_like(lsp)  # 결과를 저장할 배열

for n in range(len(lsp)):
    lsp2[n] = bark2f(f2bark(lsp[n]) + b)

# 출력 결과 확인
print(lsp2)

[110 210 311 412 513]


### 7.9 Voice and pitch changer

#### 7.9.1 PSOLA

In [12]:
# 기존에 설계한 ltp 함수 재호출 코드
import scipy.signal as signal

def ltp(sp):
    n = len(sp)
    
    # 피치 검색 범위 설정
    pmin = 50
    pmax = 200
    
    # 시간 절약을 위해 신호 제곱을 미리 계산
    sp2 = sp ** 2
    
    # E 배열 계산
    E = []
    for M in range(pmin, pmax + 1):
        e_del = sp[:n - M]
        e = sp[M:]
        e2 = sp2[M:]
        E.append(np.sum((e_del * e) ** 2) / np.sum(e2))
    
    # 최적의 피치 주기 M 찾기
    M_opt = np.argmax(E) + pmin
    
    # 피치 이득 B 계산
    e_del = sp[:n - M_opt]
    e = sp[M_opt:]
    e2 = sp2[M_opt:]
    B = np.sum(e_del * e) / np.sum(e2)
    
    return B, M_opt

In [13]:
# 파라미터 설정 및 신호 생성
sp = np.random.randn(1024)  # 임의의 신호
B, M = ltp(sp)  # 피치와 LTP 추정
sc = 0.35  # 스케일링 비율
M2 = int(round(M * sc))
# Hamming 창 생성
win = signal.hamming(2 * M)
# N 프레임으로 신호 분할
N = np.floor(len(sp) / M).astype(int)
out = np.zeros(N*M2+M)

# 각 프레임을 윈도우 처리하고 재구성(본문과 수식 처리 다르게 설정)
for n in range(N - 1):
    # 인덱스 계산
    fr1 = n * M
    to1 = fr1 + 2 * M
    
    # 인덱스가 범위를 넘지 않도록 처리
    if to1 > len(sp):
        break
    
    # 신호와 창 함수 곱하기
    seg = sp[fr1:to1] * win
    
    # 새로운 신호 위치 계산
    fr2 = n * M2
    to2 = fr2 + M
    
    # 음수 인덱스를 피하기 위해 처리
    fr2b = max(0, fr2)
    
    # 경계를 넘지 않도록 배열 범위 제한
    if to2 > len(out):
        to2 = len(out)
    
    # 신호 재구성
    out[fr2b:to2] += seg[:to2 - fr2b]

# 결과 출력
print(out)

[ 4.12491371e-02  3.99811065e-02 -2.66376554e-02 -3.62545363e-02
 -5.75504521e-02 -1.51528734e-01 -5.82409446e-02 -1.36507974e-01
 -7.88562409e-03 -5.12495518e-02  1.18856175e-01  1.03202985e-02
  8.20848350e-03 -1.83383713e-01 -7.54454091e-02 -7.44446164e-02
  2.89564404e-02  2.08690997e-01  6.91165650e-02 -3.39348510e-02
 -1.06828962e-01  4.94177293e-03  1.89451971e-02 -2.44933994e-02
  2.09431732e-01 -5.28372398e-03 -2.38200278e-02 -4.42894913e-02
  1.23217290e-01  1.31047636e-03 -1.23025517e-01 -1.04003679e-01
 -3.28024831e-01  1.09954019e-01  3.19196916e-01  4.96185679e-02
  2.72562967e-01  2.42798038e-01 -3.60936380e-01 -3.43058783e-02
 -1.88593347e-01  2.30689036e-01 -1.80511945e-03  5.21628905e-02
 -2.71867781e-01 -1.74358824e-01  8.28008036e-02  2.08360928e-01
  6.80173372e-02 -8.12046055e-02  2.03104596e-01 -4.78281395e-02
 -3.88116956e-01 -1.81228172e-01  3.00230097e-01 -4.43767731e-01
 -1.78348925e-01  5.02047730e-02 -3.19840359e-01 -4.35928641e-01
 -4.40730097e-01 -2.88216

#### 7.9.2 LSP-based method

In [14]:
# 임의로 생성한 lpc, lpc_lsp, lsp_lpc 함수 코드
from scipy.linalg import toeplitz

def lpc(signal, order):
    # 오토 상관 함수 계산
    autocorr = np.correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr) // 2:]  # 양의 지연만 사용
    
    # Toeplitz 행렬 생성
    R = toeplitz(autocorr[:order])
    r = autocorr[1:order + 1]
    
    # LPC 계수 계산 (Yule-Walker 방정식)
    lpc_coeffs = np.linalg.solve(R, r)
    
    # LPC 계수 반환 (1을 추가해 전체 계수로 반환)
    return np.concatenate(([1], -lpc_coeffs))

def lpc_lsp(a):
    p = len(a)
    A = [1]
    B = [1]

    # A와 B의 값 계산
    for k in range(1, p):
        A.append((a[k] - a[p - k]) + A[k - 1])
        B.append((a[k] + a[p - k]) - B[k - 1])

    # 다항식의 근 계산
    r1 = np.roots(A)
    r2 = np.roots(B)

    theta1 = []
    theta2 = []

    # theta1과 theta2 계산
    for k in range(p - 1):
        if np.real(r1[k]) < 0:
            theta1.append(np.pi - abs(np.arctan2(np.imag(r1[k]), np.real(r1[k]))))
        else:
            theta1.append(abs(np.arctan2(np.imag(r1[k]), np.real(r1[k]))))
        
        if np.real(r2[k]) < 0:
            theta2.append(np.pi - abs(np.arctan2(np.imag(r2[k]), np.real(r2[k]))))
        else:
            theta2.append(abs(np.arctan2(np.imag(r2[k]), np.real(r2[k]))))

    # theta 계산
    p -= 1
    theta = [0] * p
    for k in range(int(p / 2)):
        theta[k] = theta1[k * 2]
        theta[k + int(p / 2)] = theta2[k * 2]

    # 정렬하여 lsp 반환
    lsp = np.sort(theta)
    
    return lsp


def lsp_lpc(theta):
    p = len(theta)
    q = np.cos(theta)

    # f1와 f2 초기화
    f1 = np.zeros(20)
    f1[9] = 1  # MATLAB에서 f1(10)은 파이썬에서 f1[9]임
    
    f2 = np.zeros(20)
    f2[9] = 1  # MATLAB에서 f2(10)은 파이썬에서 f2[9]임

    # f1 계산
    for i in range(1, p//2 + 1):
        f1[9 + i] = -2 * q[2 * i - 2] * f1[9 + i - 1] + 2 * f1[9 + i - 2]
        for k in range(i - 1, 0, -1):
            f1[9 + k] = f1[9 + k] - 2 * q[2 * i - 2] * f1[9 + k - 1] + f1[9 + k - 2]

    # f2 계산
    for i in range(1, p//2 + 1):
        f2[9 + i] = -2 * q[2 * i - 1] * f2[9 + i - 1] + 2 * f2[9 + i - 2]
        for k in range(i - 1, 0, -1):
            f2[9 + k] = f2[9 + k] - 2 * q[2 * i - 1] * f2[9 + k - 1] + f2[9 + k - 2]

    # f1b와 f2b 계산
    f1b = np.zeros(p // 2)
    f2b = np.zeros(p // 2)

    f1b[0] = f1[10] + 1
    f2b[0] = f2[10] - 1

    for i in range(1, p // 2):
        f1b[i] = f1[9 + i + 1] + f1[9 + i]
        f2b[i] = f2[9 + i + 1] - f2[9 + i]

    # a2 계산
    a2 = np.zeros(p)
    for i in range(p // 2):
        a2[i] = 0.5 * (f1b[i] + f2b[i])
        a2[i + p // 2] = 0.5 * (f1b[(p // 2) - i - 1] - f2b[(p // 2) - i - 1])

    # 최종 LPC 계수 반환
    a = np.concatenate(([1], a2))

    return a

In [15]:
# 파라미터 설정 및 신호 생성
sp = np.random.randn(1024)  # 임의의 신호
Beta, tapA = ltp(sp)

# 피치 추출
vocA = signal.lfilter([1] + [0] * (tapA - 1) + [-Beta], [1], sp)

# LPC 분석 수행
aA = lpc(sp * signal.hamming(len(vocA)), 10)
lung = signal.lfilter(aA, [1], vocA)

# LSP 변환
lspA = lpc_lsp(aA)

# 변형
lspB = lspA / 1.1
tapB = int(round(tapA * 1.1))

# LPC로 변환
aB = lsp_lpc(lspB)

# 음성 재생성
vocB = signal.lfilter([1], aB, lung)
spB = signal.lfilter([1] + [0] * (tapB - 1) + [Beta], [1], vocB)

# 결과 출력
print(spB)

[  -0.71805505   -6.44117054  -24.73622066 ... -104.23405961 -436.60167821
 -561.32399355]
